# Task 1

In [35]:
import pandas as pd 
from math import cos, asin, sqrt

ports = pd.read_csv("ports.csv")

tracking = pd.read_csv("tracking.csv").drop_duplicates().sort_values(by='datetime', ascending=True)
#.drop_duplicates(subset=['vessel', 'lat', 'long'], keep='first')


In [36]:
# https://stackoverflow.com/a/41337005

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))

def closest_port(data, v, ports):
    pos = min(data, key=lambda p: distance(v['lat'],v['long'],p['lat'],p['long']))
    dist = distance(pos['lat'], pos['long'], v['lat'], v['long'])
    return (ports[(ports.lat == pos['lat']) & (ports.long == pos['long'])]['port'].values[0], dist)


In [62]:
speed_threshold = 1
ports_dict = ports[['lat', 'long']].to_dict('records')
ports_km_threshold = 3

started_port = 0
voyages_list = []
end_port_id = 0

for vessel in tracking.drop_duplicates(subset=['vessel'])['vessel'].values:
    for index, row in tracking[tracking.vessel == vessel].iterrows():
        if started_port == 0 and row['speed'] < speed_threshold:
            #record startport
            started_port = 1
            (begin_port_id, _) = closest_port(ports_dict, {'lat': row['lat'], 'long': row['long']}, ports)
        elif started_port == 1 and row['speed'] > speed_threshold:
            #record start date 
            started_port = 2
            begin_date = row['datetime']
        elif started_port == 2 and row['speed'] < speed_threshold:
            #record end date, end port, add to new df
            end_date = row['datetime']
            (end_port_id, closest_dist) = closest_port(ports_dict, {'lat': row['lat'], 'long': row['long']}, ports)
            if ports_km_threshold > closest_dist:
                voyages_list.append({'vessel': vessel, 'begin_date': begin_date, 'end_date': end_date, 'begin_port_id': begin_port_id, 'end_port_id': end_port_id})
                started_port = 0
        elif started_port == 0 and row['speed'] > speed_threshold and end_port_id != 0:
            begin_date = row['datetime']
            begin_port_id = end_port_id
            started_port = 2
    started_port = 0
    end_port_id = 0
voyages = pd.DataFrame(voyages_list)
voyages = voyages[voyages['begin_port_id'] != voyages['end_port_id']]
voyages

,vessel,begin_date,end_date,begin_port_id,end_port_id
0,131,2019-01-07 07:59:00,2019-02-14 01:57:00,13,48
1,131,2019-02-15 01:59:00,2019-03-02 22:07:00,48,69
2,131,2019-03-04 11:46:00,2019-03-10 01:46:00,69,29
3,131,2019-03-11 06:37:00,2019-03-20 09:05:00,29,69
4,131,2019-03-21 21:54:00,2019-04-05 20:38:46,69,11
...,...,...,...,...,...
2929,107,2019-10-15 20:49:39,2019-10-28 20:46:30,99,51
2930,107,2019-10-29 23:52:35,2019-11-09 00:49:44,51,99
2931,107,2019-11-10 05:32:07,2019-12-01 21:49:30,99,73
2932,107,2019-12-03 00:54:56,2019-12-11 23:21:49,73,72


In [56]:
# Assertions

print(len(voyages[voyages.begin_date > voyages.end_date]) == 0)

found_error = 0
for vessel in voyages.drop_duplicates(subset=['vessel'])['vessel'].values:
    prev_port = 0
    for index, row in voyages[voyages.vessel == vessel].iterrows():
        if prev_port != 0:
            if prev_port != row['begin_port_id']:
                print("Error in index "+ str(index))
                found_error = 1
        prev_port = row['end_port_id']
print(found_error == 0)

True
True


In [57]:
voyages.to_csv('voyages.csv', index=False)
